# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('../../data_sets/kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to nadyaortiza@gmail.com and will expire on April 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1495090405.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.896866     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.981655     | 6847177.773672     | 380824.455891 |

| 10        | 11       | 0.000002  | 2.381862     | 6842123.232651     | 364204.576180 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

Find what features had non-zero weight.

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [7]:
def compute_RSS(predictions, output):
    residuals = predictions - output
    RSS = (residuals**2).sum()
    return RSS

In [8]:
import numpy as np
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print np.poly1d(w) 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [9]:
print "Features choosen:"
print "x3,x4,x5,x13,x14"
print "bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above"

Features choosen:
x3,x4,x5,x13,x14
bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above


In [10]:
print_coefficients(model_all)
print model_all.coefficients.print_rows(20)

Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      | 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
def create_lasso_regression(L1_penalities, max_nonzeros=0):
    RSS_best = None
    L1_best = None
    best_model = None
    l1_penalty_min = None
    l1_penalty_max = None
    max_num_of_nnz = None
    min_num_of_nnz = None
    continue_search = True

    for l1_penalty in L1_penalities:
        current_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose=False,
                                                  l2_penalty=0., l1_penalty=l1_penalty)
        predictions = current_model.predict(validation)

        RSS = compute_RSS(predictions, validation['price'])
        print "L1 penalty (%s)\tRSS=%s" % (l1_penalty, RSS)
        if RSS_best is None or RSS < RSS_best:
            RSS_best = RSS
            L1_best = l1_penalty
            best_model = current_model
            
        if max_nonzeros: 
            current_num_nnz = current_model.coefficients['value'].nnz()
            print "\t\tNon-Zeros: %s" % current_num_nnz 
            
            if continue_search:
            
                #The largest l1_penalty that has more non-zeros than max_nonzeros 
                if current_num_nnz > max_nonzeros:
                    max_num_of_nnz = current_num_nnz
                    l1_penalty_min = l1_penalty
                else:
                    min_num_of_nnz = current_num_nnz
                    l1_penalty_max = l1_penalty
                    continue_search=False

    lasso_info ={'RSS_best': RSS_best, 'L1_best': L1_best, 'Best model': best_model,
             'l1_penalty_min': l1_penalty_min, 'l1_penalty_max': l1_penalty_max,
            'max_num_of_nnz': max_num_of_nnz, 'min_num_of_nnz': min_num_of_nnz}


    return lasso_info

In [13]:
L1_penalities =  np.logspace(1, 7, num=13)
lasso_info = create_lasso_regression(L1_penalities) 
print "Best L1 penalty (%.2f)\tRSS=%s" % (lasso_info["L1_best"], lasso_info["RSS_best"])

L1 penalty (10.0)	RSS=6.25766285142e+14
L1 penalty (31.6227766017)	RSS=6.25766285362e+14
L1 penalty (100.0)	RSS=6.25766286058e+14
L1 penalty (316.227766017)	RSS=6.25766288257e+14
L1 penalty (1000.0)	RSS=6.25766295212e+14
L1 penalty (3162.27766017)	RSS=6.25766317206e+14
L1 penalty (10000.0)	RSS=6.25766386761e+14
L1 penalty (31622.7766017)	RSS=6.25766606749e+14
L1 penalty (100000.0)	RSS=6.25767302792e+14
L1 penalty (316227.766017)	RSS=6.25769507644e+14
L1 penalty (1000000.0)	RSS=6.25776517727e+14
L1 penalty (3162277.66017)	RSS=6.25799062845e+14
L1 penalty (10000000.0)	RSS=6.25883719085e+14
Best L1 penalty (10.00)	RSS=6.25766285142e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [14]:
print "Best L1 penalty (10.00) RSS=6.25766285142e+14"

Best L1 penalty (10.00) RSS=6.25766285142e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [15]:
print "ALL non zeros weights"
best_model = lasso_info["Best model"]
print best_model.coefficients['value'].nnz() #NON-ZEROS
print best_model.coefficients.print_rows(20)
print_coefficients(best_model)

ALL non zeros weights
18
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    s

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [16]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [17]:
l1_penalty_values = np.logspace(8, 10, num=20)
lasso_info_nnz= create_lasso_regression(l1_penalty_values, max_nonzeros) 

L1 penalty (100000000.0)	RSS=6.27492659875e+14
		Non-Zeros: 18
L1 penalty (127427498.57)	RSS=6.28210516771e+14
		Non-Zeros: 18
L1 penalty (162377673.919)	RSS=6.29176689541e+14
		Non-Zeros: 18
L1 penalty (206913808.111)	RSS=6.30650082719e+14
		Non-Zeros: 18
L1 penalty (263665089.873)	RSS=6.32940229287e+14
		Non-Zeros: 17
L1 penalty (335981828.628)	RSS=6.3626814023e+14
		Non-Zeros: 17
L1 penalty (428133239.872)	RSS=6.41261198311e+14
		Non-Zeros: 17
L1 penalty (545559478.117)	RSS=6.48983455376e+14
		Non-Zeros: 17
L1 penalty (695192796.178)	RSS=6.60962217696e+14
		Non-Zeros: 17
L1 penalty (885866790.41)	RSS=6.77261520728e+14
		Non-Zeros: 16
L1 penalty (1128837891.68)	RSS=7.01046815867e+14
		Non-Zeros: 15
L1 penalty (1438449888.29)	RSS=7.37850622829e+14
		Non-Zeros: 15
L1 penalty (1832980710.83)	RSS=7.9616310964e+14
		Non-Zeros: 13
L1 penalty (2335721469.09)	RSS=8.69018172894e+14
		Non-Zeros: 12
L1 penalty (2976351441.63)	RSS=9.66925692362e+14
		Non-Zeros: 10
L1 penalty (3792690190.73)	RSS=

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [18]:
print "Best L1 penalty (%.2f)\tRSS=%s" % (lasso_info_nnz["L1_best"], lasso_info_nnz["RSS_best"]) 
print "L1 penalty_min (%.2f)\tMax_nnz=%s" % (lasso_info_nnz["l1_penalty_min"],lasso_info_nnz["max_num_of_nnz"])
print "L1 penalty_max (%.2f)\tMin_nnz=%s" % (lasso_info_nnz["l1_penalty_max"],lasso_info_nnz["min_num_of_nnz"])

Best L1 penalty (100000000.00)	RSS=6.27492659875e+14
L1 penalty_min (2976351441.63)	Max_nnz=10
L1 penalty_max (3792690190.73)	Min_nnz=6


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [19]:
l1_penalty_min = lasso_info_nnz["l1_penalty_max"]
l1_penalty_max = lasso_info_nnz["l1_penalty_min"]

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [20]:
print l1_penalty_max
print l1_penalty_min
print "L1 penalty_max (2976351441.63)  Max_nnz=10"
print "L1 penalty_min (3792690190.73)  Min_nnz=6"

2976351441.63
3792690190.73
L1 penalty_max (2976351441.63)  Max_nnz=10
L1 penalty_min (3792690190.73)  Min_nnz=6


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [21]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [22]:
# lasso_info_narrow = create_lasso_regression(l1_penalty_values) 

RSS_best=None
L1_best=None
best_model=None
for l1_penalty in l1_penalty_values:
    current_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    current_num_nnz = current_model.coefficients['value'].nnz()
    
    print "\t\tNon-Zeros: %s" % current_num_nnz 
    if current_num_nnz == max_nonzeros:
    
        predictions = current_model.predict(validation) 
        RSS = compute_RSS(predictions, validation['price'])
        print "L1 penalty (%s)\tRSS=%s" % (l1_penalty, RSS)
        if RSS_best is None or RSS < RSS_best:
            RSS_best = RSS
            L1_best = l1_penalty
            best_model = current_model


		Non-Zeros: 6
		Non-Zeros: 6
		Non-Zeros: 6
		Non-Zeros: 6
		Non-Zeros: 6
		Non-Zeros: 7
L1 penalty (3577864204.13)	RSS=1.06079953176e+15
		Non-Zeros: 7
L1 penalty (3534899006.81)	RSS=1.05599273534e+15
		Non-Zeros: 7
L1 penalty (3491933809.48)	RSS=1.05114762561e+15
		Non-Zeros: 7
L1 penalty (3448968612.16)	RSS=1.04693748875e+15
		Non-Zeros: 8
		Non-Zeros: 8
		Non-Zeros: 8
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10
		Non-Zeros: 10


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [23]:
print "1) L1 penalty (3448968612.16)	RSS=1.04693748875e+15"

1) L1 penalty (3448968612.16)	RSS=1.04693748875e+15


In [24]:
print best_model.coefficients.print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |

In [25]:
print "2) NON-ZERO: bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above"

2) NON-ZERO: bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above
